<a href="https://colab.research.google.com/github/tanakt-hub/Test/blob/main/JPMA_2022_TF_1_1_demo_(a)%E3%82%B3%E3%82%B5%E3%82%A4%E3%83%B3%E9%A1%9E%E4%BC%BC%E5%BA%A6%E3%81%AB%E3%82%88%E3%82%8B%E6%AF%94%E8%BC%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前準備

## 必要なライブラリのインストール

In [1]:
!pip install mecab-python3 fugashi
!pip install jaconv neologdn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 6.1 MB/s 
     |████████████████████████████████| 583 kB 23.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 462 kB/s 
  Created wheel for jaconv: filename=jaconv-0.3-py3-none-any.whl size=15566 sha256=e4e43014f44cd2bd41fa43221c42d28f03aca463885f9c37d2c22fa49a4349ba
  Stored in directory: /root/.cache/pip/wheels/8f/4f/c2/a2a3b14d0e94f855f4aa8887bf0267bee9ecfb8e62a9ee2d92
  Created wheel for neologdn: filename=neologdn-0.5.1-cp37-cp37m-linux_x86_64.whl size=172956 sha256=4a87841c8693fc31c2a69ac1796c1e71567df943a9e72108fd7fb0f6244d2486
  Stored in directory: /root/.cache/pip/wheels/19/15/5c/55b33d02e16129ef81313e4c86e473d6dd1cecf7317a525a9b
Successfully built jaconv neologdn


## データロードと環境構築

In [2]:
# MeCabとNEologdの設定
!apt install mecab libmecab-dev mecab-ipadic-utf8 file
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -a -y

# 環境変数でmecabrcの場所を指定
import os
os.environ['MECABRC'] = "/etc/mecabrc" 

# NEologdの展開場所を取得
import subprocess
cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
neologd_dic_dir_path = subprocess.check_output(cmd, shell=True).decode('utf-8').strip()

# 万病辞書のダウンロードと設定
!wget http://sociocom.jp/~data/2018-manbyo/data/MANBYO_201907_Dic-utf8.dic
manbyo_dic_path = 'MANBYO_201907_Dic-utf8.dic'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libmecab2 mecab-ipadic mecab-jumandic
  mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  file libmagic-mgc libmagic1 libmecab-dev libmecab2 mecab mecab-ipadic
  mecab-ipadic-utf8 mecab-jumandic mecab-jumandic-utf8 mecab-utils
0 upgraded, 11 newly installed, 0 to remove and 5 not upgraded.
Need to get 29.3 MB of archives.
After this operation, 282 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-upd

# テキストの前処理

## 関数定義

In [3]:
import jaconv
import unicodedata
import neologdn
import re

import MeCab

# MeCab辞書にNEologdと万病辞書を指定
tagger = MeCab.Tagger("-d " + neologd_dic_dir_path + " -u " + manbyo_dic_path)

# MeCabによる分かち書き＆原形変換関数を定義
def genkei(text):
  words = []
  for c in tagger.parse(text).splitlines()[:-1]:
    # surfaceに単語、featureに解析結果を格納
    # 対象が存在しない場合は無視して次へ
    try:
        surface, feature = c.split('\t')
    except:
        continue

    # 解析結果から品詞と原形を取得
    hinshi = feature.split(',')[0]
    genkei = feature.split(',')[6]

    # 原形が定義されている場合は単語を原形に置き換え
    if feature.split(',')[-1].isdigit() == False and genkei != '*':
      surface = genkei

    # 残す品詞を定義
    if hinshi in ['名詞','形容詞','助動詞', '動詞'] and surface != '*':
      words.append(surface)
    else:
      continue

  return ' '.join(words)

# 文字の正規化後に上記関数を実行する事前処理関数を定義
def preprocess(text):
  text = jaconv.normalize(text, "NFKC")
  text = unicodedata.normalize("NFKC", text)
  text = neologdn.normalize(text)

  text = re.sub(r'。$', '', text)   # 文末の。を削除
  text = re.sub(r'\d+', '0', text)  # 連続する数値を0に置換

  text = genkei(text)
  return text

## 前処理の実行

In [4]:
import pandas as pd
import pickle

# 処理対象データをDataFrameに格納
Texts = pd.read_table('https://raw.githubusercontent.com/tanakt-hub/Test/main/data/Label-y_v5.txt')

# 事前処理を実行したカラムを追加
Texts['wakati'] = Texts['text'].apply(preprocess)

# 処理結果の確認
Texts

,flg,text,wakati
0,0,患者はmethylprednisolone(静脈内、1 g、1日1回、使用理由:有害事象)、...,患者 methylprednisolone 静脈 内 0 g 0 日 0回 使用 理由 有害...
1,0,ラフチジン錠10mg「サワイ」(2錠、分2、朝食、夕食後、28日間、)、ベイスンOD錠0.3...,ラフチジン 錠 0m g サワイ 0 錠 0 朝食 夕食 後 0 日間 ベイスン OD 錠 ...
2,0,患者はfamotidine(静脈内、20 mg、2021年)、chlorphenamine ...,患者 famotidine 静脈 内 0 mg 0 年 chlorphenamine mal...
3,0,Seeplex RV15ワンステップACE検出測定キットを用いた喀痰及び咽頭スワブのマルチプ...,Seeplex RV 0 ワンステップ ACE 検出 測定 キット 用いる た 喀痰 咽頭 ...
4,0,ウイルス検査:(陰性)重症急性呼吸器症候群コロナウイルス2型、並びにエンテロウイルス、アデノ...,ウイルス検査 陰性 重症 急性呼吸 器 症候群 コロナウイルス 0 型 エンテロウイルス ア...
...,...,...,...
4323,1,倦怠感,倦怠感
4324,1,低値,値
4325,1,尿閉,尿閉
4326,1,脱水,脱水


# 入力テキストとラベル付きデータのコサイン類似度計算

## 関数定義

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from tqdm.notebook import tqdm

# TF-IDFの計算関数
def CalcTfidf(TextSet):
  fn = TfidfVectorizer(token_pattern='\\S+')  # 半角スペースのみでSplitするように再定義
  vec = fn.fit_transform(TextSet)
  return vec.astype(float)

# ラベル付きデータに新規文章を加えてTF-IDFを計算する関数
def TfidfAll(LabelDF, NewTextList):

  PreDF = pd.DataFrame(NewTextList, columns = ["text"])
  PreDF['wakati'] = PreDF['text'].apply(preprocess)
  PreDF["flg"] = 9

  ConcatDF = pd.concat([LabelDF,PreDF])
  Calctgt = ConcatDF['wakati'].tolist()

  ConcatDF["TFIDF"] = CalcTfidf(Calctgt).toarray().tolist()

  # Positive, Negative, 新規文章のDataFrameを返す
  NewDF = ConcatDF.query('flg == 9')
  PosDF = ConcatDF.query('flg == 1')
  NegDF = ConcatDF.query('flg == 0')

  return NewDF, PosDF, NegDF

# コサイン類似度を計算する関数
def cos_sim(t1, t2):
    return np.dot(t1, t2) / (np.linalg.norm(t1) * np.linalg.norm(t2))

# 比較対象の中で最大のコサイン類似度を返す関数
def max_cos_sim(Txt,TxtSet):
  x = 0
  id = 0
  for i, t in enumerate(TxtSet):
    if x < cos_sim(Txt, t):
      x = cos_sim(Txt, t)
      id = i
  return x, id

# NewDFをPosDFとNegDFに対してコサイン類似度の総当たりをする関数
def compareDF (NewDF, PosDF, NegDF):
  pos_sim = []
  neg_sim = []
  pt = []
  nt = []
  for t in tqdm(NewDF['TFIDF'], total = len(NewDF['TFIDF'])):
    pos, pid = max_cos_sim(np.array(t), np.array(PosDF['TFIDF']))
    pos_sim.append(pos)
    pt += [Pos.iat[pid,1]]

    neg, nid = max_cos_sim(np.array(t), np.array(NegDF['TFIDF']))
    neg_sim.append(neg)
    nt += [Neg.iat[nid,1]]

  # 最大のコサイン類似度とその時の比較対象テキストをそれぞれ格納する
  NewDF["Positive"] = pos_sim
  NewDF["Negative"] = neg_sim
  NewDF["Positive Text"] = pt
  NewDF["Negative Text"] = nt

  # 不要なカラムを削除
  NewDF = NewDF.drop(['flg', 'TFIDF'], axis=1)

  # PositiveとNegativeの値からPredictionを算出
  NewDF["Pred"] = NewDF[["Positive","Negative"]].idxmax(axis=1).apply(lambda x: 0 if x == "Negative" else 1)
  return NewDF

## ラベル付きデータによる検証

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

# 実行完了までに30-40分かかる
X_train, X_test, y_train, y_test = train_test_split(Texts, Texts["flg"], test_size=0.15, random_state = 0)

New, Pos, Neg = TfidfAll(X_train, X_test)
Result = compareDF(New, Pos, Neg)
Result['Label'] = y_test

display(Result)

print('\n****: Scores')
print('        F1:', f1_score(Result["Label"], Result["Pred"]))
print('    Recall:', recall_score(Result["Label"], Result["Pred"]))
print('  Accuracy:', accuracy_score(Result["Label"], Result["Pred"]))
print(' Precision:', precision_score(Result["Label"], Result["Pred"]))

  0%|          | 0/650 [00:00<?, ?it/s]

,text,wakati,Positive,Negative,Positive Text,Negative Text,Pred,Label
4220,心尖部血栓を認めた。,心尖部血栓 認める た,0.201612,0.469085,胸痛を認めた。,心尖部血栓を認めなかったため直接経口抗凝固薬による治療終了。,0,1
3428,2022/05/30、患者は劇症型心筋炎を発現した。,0 0 0 患者 劇症型 心筋炎 発現 する た,0.676901,0.493653,23:00心筋炎が発現。,2022/05/17、患者は再受診した。,1,1
2134,"再度診察を行ったところ甲状腺左築に圧痛があり,追加検査にてTSH0.006μU/mL,Fre...",診察 行う た ところ 甲状腺 左 築 圧痛 ある 追加 検査 TSH 0 0 μ U mL...,0.395434,0.268140,検査所見:2022/06/04、トロポニンTは、1.9ng/ml(上昇あり)であった。,トロポニンT:methylprednisoloneの静脈内投与から3?日後、著明に減少した、...,1,1
555,頭部CTは異常所見なく、頭痛の器質的疾患は否定された。,頭部 C，T 異常所 見る ない 頭痛 の 器質的疾患 否定 する れる た,0.274159,0.459462,頭部CTにて、腫瘍内出血を認めた。,器質的疾患なし。,0,0
2347,2021/07/24、両大腿の感覚障害と歩行障害(バランスがとれず、スムーズに一歩ずつ足が出...,0 0 0 大腿 感覚障害 歩行障害 バランス とれる ぬ スムーズ 一 歩 足 出る ない...,1.000000,0.192393,2021/07/24には、両大腿の感覚障害と歩行障害(バランスがとれず、スムーズに一歩ずつ足...,2022/05/16一日中動悸が治まらなかったが、一日で回復。,1,1
...,...,...,...,...,...,...,...,...
4019,翌日に右眼の視力低下を自覚した。,翌日 右 眼 視力低下 自覚 する た,0.736449,0.392984,接種後5日目から右眼の視力低下を自覚した。,視力の右眼は改善したが、左眼は改善を得なかった。,1,1
398,被験者にSARS-CoV2感染中に悪化した基礎疾患はなかった。,被験者 SARS CoV 0 感染中 悪化 する た 基礎疾患 ない た,0.300684,0.934470,2022/01/27、SARS-CoV-2検査(PCR)で、COVID-19陽性を示した。,SARS- CoV2感染中に悪化した基礎疾患はなかった。,0,0
4092,両側肺動脈に透亮像を認めた。,両側肺 動脈 透亮像 認める た,0.326733,0.272075,陽電子放出断層撮影:胸大動脈、鎖骨下動脈、腋窩動脈、上腕動脈、側頭動脈で過蓄積が見られた,直近の冠状動脈検査:未実施。,1,1
538,ワクチン接種以来、COVID-19の検査はしていない。,ワクチン接種 以来 COVID 0 検査 する いる ない,0.366889,0.605561,COVID-19を発現した。,ワクチン接種以降、COVID-19の検査はしていない。,0,0



****: Scores
        F1: 0.853731343283582
    Recall: 0.8218390804597702
  Accuracy: 0.8492307692307692
 Precision: 0.8881987577639752


## 入力テキストの予測

In [7]:
txt = [
  "組織脳脊髄液検査では異常が無かったが、細菌培養及びウイルス検査は異常を認めた。",
  "2022/05/23、倦怠感(障害)発現、転帰「未回復」。",
  "日付不明発熱。",
  "発現2022/04/08、心電図ST部分上昇(医学的に重要)、転帰「軽快」、「心電図ST上昇」と記述された。",
  "膀胱ポリープ(入院)、転帰「不明」。",
  "血中クレアチンホスホキナーゼ増加(入院)、発現2022/01/15 05:00、転帰「軽快」、「血液検査でCK上昇」と記載された。",
  "小腸および皮膚の病理組織を提出した。",
  "17:30頃から徐々に症状は回復した。",
  "冠攣縮性狭心症が疑われ、冠動脈造影によるアセチルコリン負荷試験を施行したが有意な所見は認めなかった。",
  "昨日尿管結石が見つかり、下記薬を服用中であった。",
  "血漿交換療法を実施して徐々に症状の改善を得たが、発症後よりはADL軽度低下した状態。",
  "新規感染であった。",
  "日中持続したが、夜就寝可能であった。",
  "患者は食欲があった。",
  "患者には、20年の喫煙歴と30年の禁煙歴があった。",
  ]

New, Pos, Neg = TfidfAll(Texts,txt)
Result = compareDF(New, Pos, Neg)

display(Result)

  0%|          | 0/15 [00:00<?, ?it/s]

,text,wakati,Positive,Negative,Positive Text,Negative Text,Pred
0,組織脳脊髄液検査では異常が無かったが、細菌培養及びウイルス検査は異常を認めた。,組織 脳脊髄液 検査 異常 無い た 細菌 培養 ウイルス検査 異常 認める た,0.456952,0.582027,異常。,脳脊髄液検査では、蛋白105 mg/dL、細胞数24/μLで、細菌培養及びウイルス検査は異常...,0
1,2022/05/23、倦怠感(障害)発現、転帰「未回復」。,0 0 0 倦怠感 障害 発現 転帰 回復,0.673594,0.482024,:2022/06/05発現、発熱(入院)、転帰「回復」(2022/06),2022/04/13症状は未回復。,1
2,日付不明発熱。,日付 不明発熱,0.247758,0.297375,日付不明急性心筋炎が発現。,日付不明退院。,0
3,発現2022/04/08、心電図ST部分上昇(医学的に重要)、転帰「軽快」、「心電図ST上昇...,発現 0 0 0 心電図 ST 部分 上昇 医学 的 重要 転帰 軽快 心電図 ST 上昇 ...,0.631427,0.431688,心電図:ST上昇を認めた。,ST上昇の改善を認めた。,1
4,膀胱ポリープ(入院)、転帰「不明」。,膀胱ポリープ 入院 転帰 不明,0.497150,0.320339,2022/06/27、発熱(入院)発現、転帰「不明」,転帰は報告されなかった。,1
5,血中クレアチンホスホキナーゼ増加(入院)、発現2022/01/15 05:00、転帰「軽快」...,血 中 クレアチンホスホキナーゼ 増加 入院 発現 0 0 0 0 0 転帰 軽快 血液検査...,0.541074,0.461335,2022/05/01 11:00発現、振戦(入院)、転帰「回復」(2022/05/04)、「...,事象の転帰は2022/06/23に軽快していた。,1
6,小腸および皮膚の病理組織を提出した。,小腸 皮膚 病理 組織 提出 する た,0.334371,0.351566,腹部コンピュータ断層撮影:小腸部分全体で炎症性の変化が認められた、注記:小腸壁の肥厚、体液貯...,病理組織学的検査は、実施されなかった。,0
7,17:30頃から徐々に症状は回復した。,0 0 頃 症状 回復 する た,0.634063,0.784758,2022/04/27頃より発熱した。,2022/04/13症状は未回復。,0
8,冠攣縮性狭心症が疑われ、冠動脈造影によるアセチルコリン負荷試験を施行したが有意な所見は認めな...,冠 攣縮性 狭心症 疑う れる 冠動脈造影 アセチルコリン 負荷 試験 施行 する た 有意...,0.412750,0.367022,冠攣縮性狭心症の可能性を考え、後日心臓カテーテル検査を再検し、冠攣縮誘発試験を施行する方針とした。,同時に冠動脈造影を施行したが、両冠動脈に有意の狭窄病変はなかった。,1
9,昨日尿管結石が見つかり、下記薬を服用中であった。,昨日 尿管結石 見つかる 下記 薬 服用 中 だ ある た,0.172224,0.307605,事象の経過は下記の通り:患者は病院に入院時、左室機能はびまん性壁運動障害、発熱、心筋逸脱酵素...,患者は併用薬を服用しなかった。,0
